### Utilizando o conceito de transações atomica no mongo
[Referencia 1](https://www.geeksforgeeks.org/python-mongodb-delete_many/)
[Referencia 2](https://pymongo.readthedocs.io/en/stable/examples/bulk.html)
[Referencia 3](https://kb.objectrocket.com/mongo-db/how-to-bulk-insert-multiple-documents-to-a-mongodb-collection-using-python-352)

In [30]:
from pymongo import MongoClient, InsertOne, DeleteOne, DeleteMany, ReplaceOne, DeleteMany


In [31]:
# create connection with local base 
mongo_url = 'mongodb://localhost:27017/'
db = MongoClient(mongo_url)
database = db['local']
collect = database['test']


In [32]:
# defining the requests
requests = [
            DeleteMany({}),
            InsertOne({"fruta": "banana"}),
            InsertOne({"fruta": "uva"}),
            DeleteOne({"fruta": "uva"}),
            InsertOne({"fruta": "cenoura"}),
            InsertOne({"fruta": "laranja"}), 
            InsertOne({"fruta": "maça"}),
            ReplaceOne({"fruta": "cenoura"}, {"fruta": "manga"})
        ]


In [33]:
# executing the requests
collect.bulk_write(requests)


In [34]:
# Query
for item in collect.find({}):
    print(item)

{'_id': ObjectId('6258095083c0254c6e77b911'), 'fruta': 'banana'}
{'_id': ObjectId('6258095083c0254c6e77b913'), 'fruta': 'manga'}
{'_id': ObjectId('6258095083c0254c6e77b914'), 'fruta': 'laranja'}
{'_id': ObjectId('6258095083c0254c6e77b915'), 'fruta': 'maça'}


In [35]:
# insert many with collection
collect.insert_many({"fruta": f"laranja {x}"} for x in range(10))


In [36]:
# update many with collection
print([x for x in collect.find({"fruta": "laranja 1"})])
collect.update_many({"fruta": "laranja 1"}, {"$set": {"fruta": "pera 1"}})
print([x for x in collect.find({"fruta": {"$regex": "pera.*"}})])


[{'_id': ObjectId('6258095183c0254c6e77b917'), 'fruta': 'laranja 1'}]
[{'_id': ObjectId('6258095183c0254c6e77b917'), 'fruta': 'pera 1'}]


In [37]:
# Query
for item in collect.find({}):
    print(item)


{'_id': ObjectId('6258095083c0254c6e77b911'), 'fruta': 'banana'}
{'_id': ObjectId('6258095083c0254c6e77b913'), 'fruta': 'manga'}
{'_id': ObjectId('6258095083c0254c6e77b914'), 'fruta': 'laranja'}
{'_id': ObjectId('6258095083c0254c6e77b915'), 'fruta': 'maça'}
{'_id': ObjectId('6258095183c0254c6e77b916'), 'fruta': 'laranja 0'}
{'_id': ObjectId('6258095183c0254c6e77b917'), 'fruta': 'pera 1'}
{'_id': ObjectId('6258095183c0254c6e77b918'), 'fruta': 'laranja 2'}
{'_id': ObjectId('6258095183c0254c6e77b919'), 'fruta': 'laranja 3'}
{'_id': ObjectId('6258095183c0254c6e77b91a'), 'fruta': 'laranja 4'}
{'_id': ObjectId('6258095183c0254c6e77b91b'), 'fruta': 'laranja 5'}
{'_id': ObjectId('6258095183c0254c6e77b91c'), 'fruta': 'laranja 6'}
{'_id': ObjectId('6258095183c0254c6e77b91d'), 'fruta': 'laranja 7'}
{'_id': ObjectId('6258095183c0254c6e77b91e'), 'fruta': 'laranja 8'}
{'_id': ObjectId('6258095183c0254c6e77b91f'), 'fruta': 'laranja 9'}


In [38]:
# remove with regex
collect.bulk_write([DeleteOne({"fruta": {"$regex": "^l.*\s\d"}})])
for item in collect.find({}):
    print(item)


{'_id': ObjectId('6258095083c0254c6e77b911'), 'fruta': 'banana'}
{'_id': ObjectId('6258095083c0254c6e77b913'), 'fruta': 'manga'}
{'_id': ObjectId('6258095083c0254c6e77b914'), 'fruta': 'laranja'}
{'_id': ObjectId('6258095083c0254c6e77b915'), 'fruta': 'maça'}
{'_id': ObjectId('6258095183c0254c6e77b917'), 'fruta': 'pera 1'}
{'_id': ObjectId('6258095183c0254c6e77b918'), 'fruta': 'laranja 2'}
{'_id': ObjectId('6258095183c0254c6e77b919'), 'fruta': 'laranja 3'}
{'_id': ObjectId('6258095183c0254c6e77b91a'), 'fruta': 'laranja 4'}
{'_id': ObjectId('6258095183c0254c6e77b91b'), 'fruta': 'laranja 5'}
{'_id': ObjectId('6258095183c0254c6e77b91c'), 'fruta': 'laranja 6'}
{'_id': ObjectId('6258095183c0254c6e77b91d'), 'fruta': 'laranja 7'}
{'_id': ObjectId('6258095183c0254c6e77b91e'), 'fruta': 'laranja 8'}
{'_id': ObjectId('6258095183c0254c6e77b91f'), 'fruta': 'laranja 9'}


In [39]:
# The first write failure that occurs (e.g. duplicate key error) aborts the remaining operations,
requests = [
    DeleteMany({}),
    InsertOne({'_id': 3, "fruta": "maça"}),
    InsertOne({'_id': 3, "fruta": "maça"}),
    InsertOne({'_id': 1, "fruta": "maça"}),
    InsertOne({'_id': 1, "fruta": "maça"}),
]
try: collect.bulk_write(requests, ordered=False)
except Exception as err: print(err)

for item in collect.find({}):
    print(item)


batch op errors occurred, full error: {'writeErrors': [{'index': 2, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 3}, 'errmsg': 'E11000 duplicate key error collection: local.test index: _id_ dup key: { _id: 3 }', 'op': {'_id': 3, 'fruta': 'maça'}}, {'index': 4, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': 1}, 'errmsg': 'E11000 duplicate key error collection: local.test index: _id_ dup key: { _id: 1 }', 'op': {'_id': 1, 'fruta': 'maça'}}], 'writeConcernErrors': [], 'nInserted': 2, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 15, 'upserted': []}
